In [10]:
from pathlib import Path
from scanpy import read_h5ad
from umap import UMAP
import pandas as pd
import anndata

h5ad_file_paths = Path('/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/V1_MTG_Data_Copies/mapped_to_v1').glob('*/processed/*.h5ad')


In [11]:
fname_to_section = {'202201281113_H1930002Cx46MTG202007104_vmsc00401' : 'H19.30.002.Cx46.MTG2.02.007.1.04',
                    '1190312477' : 'H20.30.002.Cx30.MTG.02.007.1.01.01',
                    '1190312488' : 'H20.30.002.Cx30.MTG.02.007.1.01.03',
                    '202112021529_H1930001Cx46MTG202007203_VMSC01001' : 'H19.30.001.Cx46.MTG2.02.007.2.03',
                    '202112021532_H1930001Cx46MTG202007204_vmsc00401' : 'H19.30.001.Cx46.MTG2.02.007.2.04',
                    '202112101342_H1930001Cx46MTG0202007205_VMSC01001' : 'H19.30.001.Cx46.MTG.02.02.007.2.05',
                    '202201271619_H1930002Cx46MTG202007102_VMSC01001' : 'H19.30.002.Cx46.MTG2.02.007.1.02',
                    '202201271624_H1930002Cx46MTG202007105_vmsc00401' : 'H19.30.002.Cx46.MTG2.02.007.1.05',
                    '202205161513_1174507413_VMSC02001' : 'H19.30.001.Cx56.V1.02.007.1.01.06',
                    '1185931014' : 'H20.30.001.Cx42.V1.02.007.1.01.03',
                    '1185931018' : 'H20.30.001.Cx42.V1.02.007.1.01.04',
                    '1185931027' : 'H20.30.001.Cx42.V1.02.007.1.01.06',
                    '202204211046_H1930002Cx58V10200710102_VMSC02501' : 'H19.30.002.Cx58.V1.02.007.1.01.02',
                    '202204220937_H1930002Cx58V10200710103_VMSC02501' : 'H19.30.002.Cx58.V1.02.007.1.01.03',
                    '202204251522_H1930002Cx58V10200710104_VMSC02501' : 'H19.30.002.Cx58.V1.02.007.1.01.04',
                    '202205121148_1174507405_VMSC01601' : 'H19.30.001.Cx56.V1.02.007.1.01.05',
                    '202205161055_1174507398_VMSC02501' : 'H19.30.001.Cx56.V1.02.007.1.01.04'}

In [12]:
h5ad_raws = []
for file in h5ad_file_paths:
    adata = read_h5ad(file)
    fname = file.stem
    print(fname)
    adata.uns['section'] = fname_to_section[fname]
    adata.obs['section'] = adata.uns['section']
    adata.obs['filename'] = fname
    adata.write_h5ad(file)
    h5ad_raws.append(adata)

combo_anndata = anndata.concat(h5ad_raws, uns_merge='first')
combo_anndata.obsm['X_umap'] = UMAP(n_components=2, n_neighbors = 25, metric='euclidean', min_dist=0.4).fit_transform(combo_anndata.X)

202201281113_H1930002Cx46MTG202007104_vmsc00401
1190312477
1190312488
202112021529_H1930001Cx46MTG202007203_VMSC01001
202112021532_H1930001Cx46MTG202007204_vmsc00401
202112101342_H1930001Cx46MTG0202007205_VMSC01001
202201271619_H1930002Cx46MTG202007102_VMSC01001
202201271624_H1930002Cx46MTG202007105_vmsc00401
202205161513_1174507413_VMSC02001
1185931014
1185931018
1185931027
202204211046_H1930002Cx58V10200710102_VMSC02501
202204220937_H1930002Cx58V10200710103_VMSC02501
202204251522_H1930002Cx58V10200710104_VMSC02501
202205121148_1174507405_VMSC01601
202205161055_1174507398_VMSC02501


In [14]:
#adjust the cirro for the combined file
import numpy as np

counter = 0
for ii, gb in combo_anndata.obs.groupby("section"): #ii = section name, gb = indexed chunk of dataframe
    spatial = combo_anndata.obsm['spatial'][combo_anndata.obs.section==ii,:]
    cirro_y = -(spatial[:,1] - np.mean(spatial[:,1]))
    cirro_x = (spatial[:,0] - min(spatial[:,0])) +(counter*10000)
    counter += 1
    coordinates_cirro = np.column_stack((cirro_x, cirro_y))
    combo_anndata.obsm['spatial_cirro'][np.where(combo_anndata.obs.section==ii)] = coordinates_cirro
combo_anndata.write_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/V1_MTG_Data_Copies/mapped_to_v1/Cross_areal_mapped_to_v1.h5ad')


In [91]:
# #concatenate here
# h5ad_file_paths = Path('/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/V1_MTG_Data_Copies/mapped_to_v1').glob('*/processed/*.h5ad')


# for file in h5ad_file_paths:
#     if 'updated' not in file.stem:
#         x = read_h5ad(file)
#         h5ad_raws.append(x)
# #combo_anndata.write_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/Merscope data copies/Trygve MTG V1/Great Apes Data Copies/Great_apes_72722.h5ad')


In [93]:
# ad = read_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/Merscope data copies/Trygve MTG V1/Great Apes Data Copies/Great_apes_72722.h5ad')
# ad.obs[ad.obs['cluster'].str.contains('SEAAD')]


,cluster,subclass,neighborhood,class,merscope,avg.cor,genes_detected,total_reads,prob,volume,section,filename
